In [ ]:
from math import log10
import time

from calorimeter import Calorimeter, Simulation, Layer, Electron, Spectrum

import matplotlib.pyplot as plt
import numpy as np

## Define calorimeter
Edit the code in the next cell to define your calorimeter

In [ ]:
DEADCELLFRACTION = 0.01 # in fraction of readout channels
NOISELEVEL = 5.0  # in ionisation units

# ## An example illustrating how the code works
# 
# Create the calorimeter. In this case a sampling calorimeter with interchaging
# layers of scintillator (low density, active) and lead (high density, inactive).

mycal = Calorimeter()

# Never change the density or active fraction of a layer
lead = Layer('lead', 2.0, 0.5, 0.0) # name, density, thickness (cm), active fraction
scintillator = Layer('Scin', 0.01, 0.5, 1.0)

for i in range(40):
    mycal.add_layers([lead, scintillator])

Run a single particle through calorimeter with tracing enabled and draw the result

In [ ]:
sim = Simulation(mycal)

ionisations, cal_with_traces = sim.simulate_with_tracing(Electron(0.0, 100.0), deadcellfraction=DEADCELLFRACTION)
# Draw with particle traces
fig, ax = plt.subplots(figsize=(14, 6))
ax = cal_with_traces.draw(ax=ax, show_traces=True)
plt.tight_layout()
plt.show()

mycal.reset()

In [ ]:
CALIBRATIONERROR = 0.005 # in fraction of ionisation

thickness = sum([vol.layer.get_thickness() for vol in mycal.volumes(active=False)])
active_layers = len(mycal.volumes(active=True))

print('--- Calorimeter summary ---')

print(f'Calorimeter created with total thickness {thickness} cm, of which {active_layers} layers are active.')
if thickness > 40.0:
    print('ERROR: The calorimeter is too thick! 40cm is the maximum allowed.')
print(f'Noise level is {NOISELEVEL} ionisations per layer.')
print(f'Calibration error is {CALIBRATIONERROR*100:.1f} % per particle.')
print(f'The fraction of dead cells is {DEADCELLFRACTION*100:.2f} %.')

COST_PER_LAYER = 1000  # in arbitrary units
COST_SCINTILLATOR = 1000  # scintillator layers are more expensive
COST_DEADCELL_PENALTY = 2000  # penalty per dead cell in arbitrary units
COST_NOISE_PENALTY = 50  # penalty per unit of noise in arbitrary units

scintillator_thickness = sum([vol.layer.get_thickness() for vol in mycal.volumes(active=True)])
scintillator_cost = scintillator_thickness * COST_SCINTILLATOR
deadcell_penalty = -log10(DEADCELLFRACTION + 1e-6)
noise_penalty = max((11-NOISELEVEL), 1)  # less noise is better
total_cost = active_layers*deadcell_penalty*noise_penalty*COST_PER_LAYER + scintillator_cost
print(f'The total cost of the calorimeter is estimated to be {total_cost:.0f} in arbitrary units.')

In [ ]:
start_time = time.time()

calibration_energy = 10.0  # GeV

# Create spectrum and generate particles with discrete energy
spectrum = Spectrum(particle_type=Electron)
electrons = spectrum.discrete(n_particles=250, energy=calibration_energy)

ionisations = sim.simulate_sample(electrons, deadcellfraction=DEADCELLFRACTION)

# Add noise
ionisations += np.random.normal(0.0, NOISELEVEL, ionisations.shape)

meanionisations = np.mean(ionisations, axis=0)
rmsionisations = np.std(ionisations, axis=0)
energies = np.sum(ionisations, axis=1)

# Apply calibration error
energies *= np.random.normal(1.0, CALIBRATIONERROR, energies.shape)

rel_resolution = np.std(energies)/np.mean(energies)

end_time = time.time()
elapsed_time = end_time - start_time
print(f'Simulation took {elapsed_time/250:.4f} seconds per particle')

print(f'Relative resolution at the energy of {calibration_energy} GeV is {rel_resolution:.3f}')

calibration_factor = np.mean(energies)/calibration_energy
print(f'Calibration factor is {calibration_factor:.0f} ionisations per GeV')

In [ ]:
fig = plt.figure(figsize=(15,5))
ax1, ax2, ax3 = fig.subplots(1, 3)
zcors = [vol.z for vol in mycal.volumes(active=True)]
for i in ionisations:
    ax1.plot(zcors, i)
ax1.set_xlabel(r'$x$ [cm]')
ax1.set_ylabel(r'ionisation')

ax2.errorbar(zcors, meanionisations, yerr=rmsionisations)
ax2.set_xlabel(r'$x$ [cm]')
ax2.set_ylabel(r'ionisation (mean and spread)')

ax3.hist(energies)

plt.tight_layout()
plt.show()

In [ ]:
# Create a spectrum of electrons
spectrum = Spectrum(particle_type=Electron)
electrons = spectrum.spectrum(n_particles=1000)

ionisations = sim.simulate_sample(electrons, deadcellfraction=DEADCELLFRACTION)

# Add noise
ionisations += np.random.normal(0.0, NOISELEVEL, ionisations.shape)

meanionisations = np.mean(ionisations, axis=0)
rmsionisations = np.std(ionisations, axis=0)
energies = np.sum(ionisations, axis=1)

# Apply calibration error
energies *= np.random.normal(1.0, CALIBRATIONERROR, energies.shape)

true_energies = np.array([e.energy for e in electrons])
residual = (true_energies - energies/calibration_factor)/true_energies


In [ ]:
# Bin the true energies into 10 bins and calculate RMS of residuals in each bin
n_bins = 10
bin_edges = np.linspace(3, 50, n_bins + 1)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
bin_rms = []

for i in range(n_bins):
    mask = (true_energies >= bin_edges[i]) & (true_energies < bin_edges[i + 1])
    if np.sum(mask) > 0:
        bin_rms.append(np.std(residual[mask]))
    else:
        bin_rms.append(0)

print(f'The average energy resolution is {np.std(residual):.3f}.')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
ax1.hist(true_energies, bins=30, edgecolor='black', alpha=0.7)
ax1.set_xlabel('True Energy (GeV)')
ax1.set_ylabel('Count')
ax1.set_title('Distribution of True Energies')
ax1.grid(True, alpha=0.3)
ax2.plot(bin_centers, bin_rms, 'o-', markersize=8)
ax2.set_xlabel('True Energy (GeV)')
ax2.set_ylabel('RMS of Residuals')
ax2.set_title('Relative Energy Resolution vs True Energy')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()